# Modul #4 üçün Tapşırıqlar: 1-ci Hissə

In [1]:
# Lazım olan kitabxanalar
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.datasets import make_regression, make_classification
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import warnings

In [2]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action = 'ignore')
np.random.seed(seed = 42)

In [3]:
def generate_data(regression = True, n_observations = None, n_features = None, n_missing = None):
    if regression:
        X, Y = make_regression(n_samples = n_observations, n_features = n_features, random_state = 42)
    else:
        X, Y = make_classification(n_samples = n_observations, n_features = n_features, random_state = 42)

    column_names = [f'feature_{x}' for x in range(1, n_features + 1)]

    X = pd.DataFrame(data = X)
    Y = pd.Series(data = Y, name = 'target').to_frame()

    X.columns = column_names

    rock = ('Rock ' * 25).split()
    jazz = ('Jazz ' * 25).split()
    metal = ('Metal ' * 25).split()
    lyric = ('Lyric ' * 25).split()

    X[f'feature_{n_features + 1}'] = rock + jazz + metal + lyric

    data_frame = pd.concat(objs = [X, Y], axis = 1)

    assert data_frame.shape[0] == n_observations
    assert data_frame.shape[1] == n_features + 2

    column_names = column_names + [column for column in X.columns.tolist() if column not in column_names]

    for feature in column_names:
        data_frame.loc[data_frame.index.isin(values = data_frame[feature].sample(n = n_missing).index), feature] = np.nan

    data_frame = data_frame.sample(frac = 1.0, random_state = 42, ignore_index = True)

    return data_frame

In [4]:
# Sinifləndirmə datasetin yaradılması
df = generate_data(regression = False, n_observations = 100, n_features = 10, n_missing = 50)

# İlk beş sətrin göstərilməsi
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,target
0,NaN,NaN,NaN,NaN,-0.219101,NaN,NaN,NaN,NaN,0.235615,Lyric,0
1,NaN,NaN,NaN,0.703852,NaN,-0.727137,0.620672,-1.335344,NaN,0.177701,NaN,1
2,NaN,2.304500,1.83991,1.339702,NaN,NaN,0.444263,1.159330,NaN,-0.360966,Metal,1
3,NaN,NaN,NaN,0.211646,-0.478749,0.222134,NaN,NaN,1.255756,NaN,NaN,1
4,NaN,-0.713525,NaN,NaN,NaN,NaN,-0.525755,0.150394,-2.123896,-0.759133,NaN,1


In [5]:
# Asılı olmayan dəyişənlərin filterlənməsi
X = df.drop(columns = 'target')

# Asılı olan dəyişənin filterlənməsi
Y = df.target

# Asılı olmayan dəyişənlərin və asılı olan dəyişənin train & test setə bölünməsi
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [6]:
# Ən çox təkrarlanan dəyər və median dəyər üçün əvəzləyicinin yaradılması
mode_imputer = SimpleImputer(strategy = 'most_frequent')
median_imputer = SimpleImputer(strategy = 'median')

# Cateqorik dəyişəndə olan boş xanaların ən çox təkrarlanan dəyərlə əvəzlənməsi
X_train_cat = mode_imputer.fit_transform(X = X_train.select_dtypes(include = 'object'))
X_test_cat = mode_imputer.transform(X = X_test.select_dtypes(include = 'object'))

# Numerik dəyişənlərdə olan boş xanaların median dəyərlə əvəzlənməsi
X_train_num = median_imputer.fit_transform(X = X_train.select_dtypes(include = 'number'))
X_test_num = median_imputer.transform(X = X_test.select_dtypes(include = 'number'))

In [7]:
# Şkala dəyişəninin yaradılması
scaler = RobustScaler()

# Numerik dəyişənlərin train və test setdə eyni şkalaya salınması
X_train_num_scaled = scaler.fit_transform(X = X_train_num)
X_test_num_scaled = scaler.transform(X = X_test_num)

# Dəyişənlərin adlandırılması
X_train_num_scaled = pd.DataFrame(data = X_train_num_scaled, columns = [f'feature_{x}' for x in range(1, X_train_num_scaled.shape[1] + 1)])
X_test_num_scaled = pd.DataFrame(data = X_test_num_scaled, columns = [f'feature_{x}' for x in range(1, X_test_num_scaled.shape[1] + 1)])

In [8]:
# Kateqorik dəyişən dəyərlərinin numerik dəyişənə çevirmək üçün dəyişənin yaradılması
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

# Kateqorik dəyişən dəyərlərinin train və test setə görə numerik dəyişənə çevrilməsi
X_train_cat_ohe = ohe.fit_transform(X = X_train_cat)
X_test_cat_ohe = ohe.transform(X = X_test_cat)

# Dəyişənlərin adlandırılması
X_train_cat_ohe = pd.DataFrame(data = X_train_cat_ohe, columns = list(ohe.categories_[0]))
X_test_cat_ohe = pd.DataFrame(data = X_test_cat_ohe, columns = list(ohe.categories_[0]))

In [9]:
# Numerik və kateqorik dəyişənlərin birləşdirilməsi
X_train_processed = pd.concat(objs = [X_train_num_scaled, X_train_cat_ohe], axis = 1)
X_test_processed = pd.concat(objs = [X_test_num_scaled, X_test_cat_ohe], axis = 1)

# Assert ilə train və test setin asılı olmayan dəyişənlərinin sayının test olunması
assert X_train_processed.shape[1] == 14
assert X_test_processed.shape[1] == 14

# Hər iki data frame-in ilk beş sətrinin göstərilməsi
display(X_train_processed.head(), X_test_processed.head())

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,Jazz,Lyric,Metal,Rock
0,-9.044682,0.000000,0.000000,-0.636285,0.00000,-0.141066,0.0,0.000000,-38.325427,0.000000,0.0,0.0,1.0,0.0
1,10.111672,0.000000,-1.548907,0.000000,0.00000,0.070469,0.0,0.000000,0.000000,-102.950685,0.0,0.0,1.0,0.0
2,0.000000,-137.468478,0.000000,0.000000,0.00000,0.315229,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0
3,0.000000,239.693927,0.000000,1.704238,-0.14619,-0.184534,0.0,0.322354,-22.960839,67.494411,0.0,0.0,1.0,0.0
4,7.678358,-331.675226,-0.721214,-1.294308,0.00000,0.000000,0.0,-0.786206,0.000000,-19.182405,0.0,1.0,0.0,0.0


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,Jazz,Lyric,Metal,Rock
0,-14.962052,0.000000,1.055162,-0.280565,-0.843951,0.685088,0.000000,0.000000,-36.734057,0.000000,1.0,0.0,0.0,0.0
1,-5.245212,79.104188,-0.379354,0.000000,0.000000,0.000000,59.909219,0.134300,0.000000,0.000000,1.0,0.0,0.0,0.0
2,4.464002,0.000000,-1.608658,0.000000,0.000000,0.000000,-34.729034,0.000000,26.728122,0.000000,0.0,1.0,0.0,0.0
3,0.000000,0.000000,0.482849,0.000000,0.000000,-1.434038,132.172375,0.000000,0.000000,77.977972,0.0,0.0,1.0,0.0
4,0.000000,-12.226441,-0.600729,0.343727,0.891277,0.000000,0.000000,0.708216,0.000000,0.000000,0.0,0.0,1.0,0.0


***
### Tapşırıq #1
- **LogisticRegression** alqoritmasından istifadə edərək model dəyişənin yaradılması (max_iter = 500 parametrindən istifadə et)
- **SelectFromModel()** filter metodundan istifadə edərək ən vacib dəyişənləri seç
- Asılı olmayan train və test setdə seçilmiş ən vacib dəyişənləri saxla
- Seçilmiş ən vacib train dəyişənləri ilə modeli qur

In [10]:
from sklearn.feature_selection import SequentialFeatureSelector

# LogisticRegression() algoritmasından istifadə edərək alqoritmanın qurulması
model = LogisticRegression(max_iter = 500, random_state = 1)

rskf = RepeatedStratifiedKFold(random_state = 1)

# SelectFromModel() metodu ilə ən vacib dəyişənlərin seçilməsi
feature_selection = SequentialFeatureSelector(estimator = model, scoring = 'accuracy', cv = rskf, n_jobs = -1)
feature_selection.fit(X = X_train_processed, y = y_train)

# Train və test setd ən vacib dəyişənlərin saxlanılması
X_train_processed_selected = feature_selection.transform(X = X_train_processed)
X_test_processed_selected = feature_selection.transform(X = X_test_processed)

# Modelin seçilən dəyişənlərlə qurulması
model.fit(X_train_processed_selected, y_train)

LogisticRegression(max_iter=500, random_state=1)

- Alqoritma tərəfindən seçilmiş ən vacib dəyişənləri göstər

In [12]:
# Seçilən dəyişənlərin selected_features dəyişənində saxlanılması
selected_features = feature_selection.get_feature_names_out()

# Seçilən dəyişənlərin göstərilməsi
selected_features

array(['feature_1', 'feature_2', 'feature_3', 'feature_7', 'Jazz',
       'Lyric', 'Rock'], dtype=object)

***
### Tapşırıq #2
- Modelin dəqiqliyini train və test setə görə hesabla

In [13]:
# Modelin performansının train və test setə görə hesablanması
train_accuracy = model.score(X = X_train_processed_selected, y = y_train)
test_accuracy = model.score(X = X_test_processed_selected, y = y_test)

# Modelin performansının train və test setə görə göstərilməsi
print(f'Model performance for train set: {train_accuracy:.0%}')
print(f'Model performance for test set: {test_accuracy:.0%}')

Model performance for train set: 89%
Model performance for test set: 65%


- **summary_df** Pandas DataFrame data strukturunu yarat
- summary_df-də **Feature** sütununu yarat və ən vacib dəyişənlərin seçilməsi zamanı istifadə olunan dəyişən adlarını bu sütuna ver
- summary_df-də **Selected** sütununu yarat və ən vacib dəyişənlərin seçilməsi alqoritması tərəfindən təyin olunan bulean dəyərləri bu sütuna ver

In [14]:
# Pandas DataFrame data strukturunun yaradılması
summary_df = pd.DataFrame()

# Feature sütununun yaradılması və ən vacib dəyişənlərin seçilməsi zamanı istifadə olunan dəyişən adlarının verilməsi
summary_df['feature'] = X_train_processed.columns
# Hər bir dəyişən üçün ən vacib dəyişənlərin seçilməsi alqoritması tərəfindən təyin olunan bulean dəyərlərin verilməsi
summary_df['selected'] = feature_selection.get_support()
# DataFrame-in göstərilməsi
summary_df

,feature,selected
0,feature_1,True
1,feature_2,True
2,feature_3,True
3,feature_4,False
4,feature_5,False
5,feature_6,False
6,feature_7,True
7,feature_8,False
8,feature_9,False
9,feature_10,False


***
### Tapşırıq #3
- Qurulan modelə çarpaz validasiya tətbiq edərək ortalama, maksimum, minimum çarpaz validasya və standard yayılma dəyərlərini hesabla (scoring = 'accuracy' parametrindən istifadə et)

In [16]:
# Modelin çarpaz validasiya olunması
cross_validation = cross_val_score(estimator = model, X = X_train_processed, y = y_train, scoring = 'accuracy', cv = rskf, n_jobs = -1)

# Ortalama, maksimum, minimum çarpaz validasya və standard yayılma dəyərlərinin hesablanması
max_score = cross_validation[cross_validation.argmax()]
min_score = cross_validation[cross_validation.argmin()]
avg_score = cross_validation.mean()
std_score = cross_validation.std()

# Ortalama, maksimum, minimum çarpaz validasya və standard yayılma dəyərlərinin göstərilməsi
print(f'Average cross validation score: {avg_score:.0%}')
print(f'Maximum cross validation score: {max_score:.0%}')
print(f'Minimum cross validation score: {min_score:.0%}')
print(f'Standard deviation: {std_score:.2f}')

Average cross validation score: 80%
Maximum cross validation score: 100%
Minimum cross validation score: 50%
Standard deviation: 0.11
